<center><font size = "10"> Week 3 - Ion Channels <center>
<center><font size = "8">Tutorial 01: data files<center>

<font size = "3"><font color = "blue">In this tutorial you will learn:
    
<font size = "3"><font color = "blue">- to work and explore ion channel files 

<font size = "3"><font color = "blue">- to plot data from ion channel files
    
<font size = "3"><font color = "blue">- to fit curves to data 

## 1. Open ion channel data files and plot 

<font size = "3">Take a look at the data files from this tutorial with the extension .NWB (Neurodata Without Borders). This is a specific format to store cellular-based neurophysiology data from a single experimental session (you can find more information [here](https://nwb-schema.readthedocs.io/en/latest/).
    
<font size = "3">There are several ways to open a file like that: [PyNWB](https://pynwb.readthedocs.io/en/stable/), [HDFview](https://www.hdfgroup.org/downloads/hdfview/) or Python. You can see below an example of one of the files open with HDFview.

 <br><img src="HDFViewer_IonFileDisplay.png" width="800" height="400">   


<font size = "3">Now let's see how to open and explore the file with Python

In [ ]:
# Open data with python
import h5py

data_path = 'files/rCell2148.nwb'

data = h5py.File(data_path, 'r')
list(data.keys())

In [ ]:
# Explore the data file
# To see all the possible methods:
# type data[('stimulus')]. and press tabular key --> this will display a list with all the possibilities

print(data[('stimulus')].values)

print(data[('stimulus')].keys())

print (data[('stimulus/presentation/')].values)

print (data[('stimulus/presentation/')].keys())   # and so on...

print (data[('analysis')].values)  # this should be empty

In [ ]:
# Open the file
data_path = 'files/rCell2148.nwb'
nwbFile = data_path
open_data = h5py.File(nwbFile, 'r')

# Select on strings what do we want for ploting
string_data = '/acquisition/timeseries/VRest/repetitions/repetition1/data'

# safe on different variables
data = open_data[(string_data)]
data

# 2. Plotting data.

<font size = "3">Now that we know how to see what is inside the file, we can plot some data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plotDataFiles(nwbFile, protocolName, repID, byTraces = False):
    ''' Function .....
    :param nwbFile: the data path to file
    :param protocolName: protocol name defined in the data file
    :param repID: protocol repetition number
    :param byTraces: if true, represent each trace in one figure'''
    
    # Open the file
    open_data = h5py.File(nwbFile, 'r')
    
    # Select on strings what do we want for ploting
    string_data = '/acquisition/timeseries/'+ protocolName + '/repetitions/repetition' + repID +'/data'
    string_xinterval = '/acquisition/timeseries/'+ protocolName + '/repetitions/repetition' + repID +'/x_interval'
    string_xstart = '/acquisition/timeseries/'+ protocolName + '/repetitions/repetition' + repID +'/x_start'
    
    # safe on different variables
    data = open_data[(string_data)].value
    x_interval = open_data[(string_xinterval)].value
    x_start = open_data[(string_xstart)].value
    
    # create time vector in ms
    nRow, nCol = data.shape
    x_end = x_start[0] + x_interval[0]*(float(nRow - 1))
    time = np.linspace(x_start[0], x_end, nRow)*1000
    
    # We can plot the data in two ways:
    # way 1: all the traces in one figure
    if byTraces == False:
        plt.figure()
        plt.title('%s protocol' %protocolName)
        plt.ylabel('voltage traces (mV)')
        plt.xlabel('time (ms)')
        plt.plot(time, data, 'b')
        plt.show()
    else:
        # way 2: each trace in different figures
        data_t = np.transpose(data)
        i = 0
        for trace in data_t:
            plt.figure()
            plt.title('%s protocol, trace %s' %(protocolName,i))
            plt.ylabel('voltage traces (mV)')
            plt.xlabel('time (ms)')
            plt.plot(time, trace, 'b')
            i = i + 1
        plt.show()   

In [ ]:
# Call the previous function and plot the Activation curves

plotDataFiles(data_path, 'Deactivation', '2', byTraces=False)

# 3. Curve fitting

<font size = "3">Curve fitting, also known as regression analysis, is used to find the "best fit" line or curve for a series of data points. Most of the time, the curve fit will produce an equation that can be used to find points anywhere along the curve. 
    
<font size = "3">Here you will find two examples.

### Example 1: linear fitting

In [ ]:
dat1_path = 'files/stLine1.dat'
dat2_path = 'files/stLine2.dat'
dat3_path = 'files/stLine3.dat'

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def curveFittingStLine(fileName, fitting_values = False):
    '''This function computes a linear regression for the data in fileName'''
    # Open the file and separate the columns in x and y
    df = pd.read_table(fileName, sep='\s+', header=None)
    x = df[0]
    y = df[1]
    
    # Fitting function
    gradient, intercept, r_value, p_value, std_err = stats.linregress(x,y) 
    # y = gradient*x + intercept
    # gradient = slope of the regression line; float
    # intercept = intercept of the regression line; float
    # r_value = correlation coefficient; float
    # p_value = two-sided p-value for a hypothesis test whose null hypothesis is that the slope is zero; float
    # std_err = standard error of the estimated gradient; float
    
    # Print fitting parameters result, if we want
    # Gradient and intercept are the parameters to be computed
    # You can quantify the quality of the fitting
    # by comparing r, p and std
    if fitting_values == True:
        print ('gradient =', gradient)
        print ('intercept =', intercept)
        print ('r_value =', r_value)
        print ('p_value =', p_value)
        print ('std_err =', std_err)
    
    # Prepare predicted line for plotting 
    mn=np.min(x)
    mx=np.max(x)
    x1=np.linspace(mn,mx,500)
    y1=gradient*x1+intercept
    
    # Plot data and regresion line
    plt.plot(x,y,'xk', x1, y1, '-r')
    plt.show()

In [ ]:
curveFittingStLine(dat1_path, fitting_values = True)
curveFittingStLine(dat2_path, fitting_values = True)
curveFittingStLine(dat3_path, fitting_values = True)

### Example 2: exponential fitting

In [ ]:
dat4_path = 'files/singleExp1.dat'
dat5_path = 'files/singleExp2.dat'
dat6_path = 'files/singleExp3.dat'

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

def func_exp(x, tau):
    return 1 - np.exp(-x/tau)

def curveFittingSingleExp(fileName):
    '''This function computes an exponential fitting for the data in fileName'''
    # Open the file and separate the columns in x and y
    df = pd.read_table(fileName, sep='\s+', header=None)
    x_data = df[0]
    y_data = df[1]
    
    # In this case, we want to compute TAU = popt[0]
    # and to quantify the fitting quality we want to know perr
    popt, pcov = curve_fit(func_exp, x_data, y_data)
    perr = np.sqrt(np.diag(pcov))
    
    plt.figure(figsize=(10,6))
    plt.plot(x_data, y_data, 'xk', label='original data' )
    plt.plot(x_data, func_exp(x_data, popt[0]), '-r',label='fit: tau=%.3f, error=%.3f' %(popt[0],perr))
    plt.legend()
    plt.show()

In [ ]:
curveFittingSingleExp(dat4_path)
curveFittingSingleExp(dat5_path)
curveFittingSingleExp(dat6_path)